In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install nltk


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

def preprocess_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words and word not in string.punctuation]
    
    return tokens

# Sample text for training the CBOW model (multiple sentences)
text = """Python is a high-level, interpreted programming language known for its simplicity and readability.
It supports multiple programming paradigms, including procedural, object-oriented, and functional programming.
One of Python's significant advantages is its vast standard library, which provides modules and functions for a wide range of tasks, 
from web development to data analysis. Due to powerful libraries like NumPy and pandas, Python is widely used in scientific computing,
artificial intelligence, machine learning, and data visualization. The language is designed with intuitive syntax, making it an excellent choice 
for both beginners and experienced developers. Python also boasts a strong community, contributing to a rich ecosystem of third-party packages 
available through the Python Package Index (PyPI). Furthermore, Python is cross-platform, allowing it to run on various operating systems 
such as Windows, macOS, and Linux without requiring significant changes to the code."""

# Preprocess the text
tokens = preprocess_text(text)








[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sushm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sushm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Create training data with context and target word
training_data = []
context_size = 2 # Adjust the context size as needed

for i in range(context_size, len(tokens) - context_size):
    context = tokens[i - context_size : i] + tokens[i + 1 : i + 1 + context_size]
    target = tokens[i]
    training_data.append(context + [target])  # Combine context and target into a single list

In [5]:
# Define the CBOW model using Word2Vec
model_cbow = Word2Vec(sentences=[tokens], vector_size=500, window=context_size, sg=0, min_count=1, workers=4)

# Display the vocabulary
print("Vocabulary:")
print(model_cbow.wv.key_to_index)  # For Gensim 4.0+


Vocabulary:
{'python': 0, 'programming': 1, 'language': 2, 'significant': 3, 'data': 4, 'analysis': 5, 'wide': 6, 'range': 7, 'tasks': 8, 'web': 9, 'development': 10, 'powerful': 11, 'due': 12, 'modules': 13, 'libraries': 14, 'like': 15, 'numpy': 16, 'pandas': 17, 'widely': 18, 'used': 19, 'functions': 20, 'library': 21, 'provides': 22, 'paradigms': 23, 'interpreted': 24, 'known': 25, 'simplicity': 26, 'readability': 27, 'supports': 28, 'multiple': 29, 'including': 30, 'changes': 31, 'procedural': 32, 'functional': 33, 'one': 34, 'advantages': 35, 'vast': 36, 'standard': 37, 'scientific': 38, 'code': 39, 'artificial': 40, 'run': 41, 'packages': 42, 'available': 43, 'package': 44, 'index': 45, 'pypi': 46, 'furthermore': 47, 'allowing': 48, 'various': 49, 'intelligence': 50, 'operating': 51, 'systems': 52, 'windows': 53, 'macos': 54, 'linux': 55, 'without': 56, 'requiring': 57, 'ecosystem': 58, 'rich': 59, 'contributing': 60, 'community': 61, 'machine': 62, 'learning': 63, 'visualization

In [17]:
# Train the CBOW model
model_cbow.train(training_data, total_examples=len(training_data), epochs=10)

# Sample test data with known vocabulary
test_data = [
    (["python", "language"], "linux"),  # Context: python, language → Target: syntax
    (["widely", "used"], "artifical"),   # Context: widely, used → Target: developers
    (["data", "analysis"], "pandas"),      # Context: data, analysis → Target: pandas
    (["web", "development"], "django"),    # Context: web, development → Target: django
    (["intuitive", "syntax"], "readability"),  # Context: intuitive, syntax → Target: readability
    (["machine", "learning"], "artificial"),  # Context: machine, learning → Target: artificial
    (["libraries", "like"], "numpy"),       # Context: libraries, like → Target: numpy
    (["programming", "languages"], "python"),  # Context: programming, languages → Target: python
    (["powerful", "libraries"], "numpy"),    # Context: powerful, libraries → Target: numpy
    (["strong", "community"], "supporting")  # Context: strong, community → Target: supporting
]

# Variables to calculate accuracy
correct_predictions = 0
total_predictions = len(test_data)


In [22]:
# Predict and compare
for context, actual_target in test_data:
    try:
        # Check if all words in context are present in the model's vocabulary
        if all(word in model_cbow.wv.key_to_index for word in context):
            predicted_word = model_cbow.wv.most_similar(positive=context, topn=2)
            predicted_word_text = predicted_word[0][0]  # Get the most similar word
            print(f"Context: {context}, Predicted: {predicted_word_text}, Actual: {actual_target}")
            if predicted_word_text == actual_target:
                correct_predictions += 1
        else:
            print(f"One or more context words not in vocabulary: {context}")
    except KeyError as e:
        print(f"KeyError: {e} - Context: {context}")


Context: ['python', 'language'], Predicted: linux, Actual: linux
Context: ['widely', 'used'], Predicted: artificial, Actual: artifical
Context: ['data', 'analysis'], Predicted: choice, Actual: pandas
Context: ['web', 'development'], Predicted: designed, Actual: django
Context: ['intuitive', 'syntax'], Predicted: advantages, Actual: readability
Context: ['machine', 'learning'], Predicted: artificial, Actual: artificial
Context: ['libraries', 'like'], Predicted: powerful, Actual: numpy
One or more context words not in vocabulary: ['programming', 'languages']
Context: ['powerful', 'libraries'], Predicted: numpy, Actual: numpy
Context: ['strong', 'community'], Predicted: various, Actual: supporting


In [23]:
# Calculate accuracy
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Accuracy of the CBOW model: {accuracy:.2f}")


Accuracy of the CBOW model: 0.90
